## Initial setup

source is https://github.com/DataTalksClub/llm-zoomcamp/tree/main/05-best-practices

In [1]:
# We should pull and run a docker container with Elasticsearch 8.9.0 or higher in order to use reranking based on RRF algorithm
# run in VS code terminal:
'''
docker run -it \
    --rm \
    --name elasticsearch \
    -m 4GB \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.17.6
''';

In [2]:
%pip install sentence_transformers elasticsearch===8.17.0  --q

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install tqdm --q

In [4]:
!pip install ipywidgets --q

In [5]:
import json
import pandas as pd
from tqdm.auto import tqdm
# from tqdm.notebook import tqdm
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch

In [6]:
# !wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/05-best-practices/documents-with-ids.json
# here we need to get the link to a raw file, otherwise it will download HTML only

In [7]:
with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [8]:
documents[0] # we downloaded FAQ document pre-parsed and stored as json

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [9]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)
model 
# Transformer({'max_seq_length': 512, 'do_lower_case': False, 'architecture': 'BertModel'})
# Pooling({'word_embedding_dimension': 384 ...

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

## Encode text and question vectors and create ES index

In [10]:
# we would need question, text and qt - both question and text vectors for similarity search

for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)

  0%|          | 0/948 [00:00<?, ?it/s]

In [11]:
#documents[0] # to see our vectors added


In [12]:
# connecting to Elasticsearch running in docker container

# es_client = Elasticsearch('http://localhost:9200') 
# es_client # this does not work - requires ES 9 headers....

# Connect with compatibility headers for ES 8.x
from elasticsearch import Elasticsearch

es_client = Elasticsearch(
    ["http://localhost:9200"],
    # Force ES 8 compatibility
    headers={
        "Accept": "application/vnd.elasticsearch+json; compatible-with=8",
        "Content-Type": "application/json"
    }
)

print(es_client.info())
# it gave an error earlier due to version 8 incompatibility...

{'name': 'b5c29c66594e', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'SvluaMxJQaKB0_I9CDSv7w', 'version': {'number': '8.17.6', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7', 'build_date': '2025-04-30T14:07:12.231372970Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [13]:
# create elasticsearch index

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            }
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [14]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

## Hybrid search example

In [15]:
course = "data-engineering-zoomcamp"

In [16]:
query = 'I just discovered the course. Can I still join it?'

In [22]:
v_q = model.encode(query)

In [19]:
knn_query = {
    "field": "text_vector",
    "query_vector": v_q,
    "k": 5,
    "num_candidates": 10000,
    "boost": 0.5,
    "filter": {
        "term": {
            "course": course
        }
    }
}

In [20]:
keyword_query = {
    "bool": {
        "must": {
            "multi_match": {
                "query": query,
                "fields": ["question^3", "text", "section"],
                "type": "best_fields",
                "boost": 0.5,
            }
        },
        "filter": {
            "term": {
                "course": course
            }
        }
    }
}

In [23]:
response = es_client.search(
    index=index_name,
    query=keyword_query,
    knn=knn_query,
    size=5
)

In [25]:
# response["hits"]["hits"]
# '_score': 36.424633,
#   '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks

## Hybrid search pipeline

In [27]:
# !wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/05-best-practices/ground-truth-data.csv

In [30]:
df_ground_truth = pd.read_csv('ground-truth-data.csv')
df_ground_truth.head()

,question,course,document
0,When does the course begin?,data-engineering-zoomcamp,c02e79ef
1,How can I get the course schedule?,data-engineering-zoomcamp,c02e79ef
2,What is the link for course registration?,data-engineering-zoomcamp,c02e79ef
3,How can I receive course announcements?,data-engineering-zoomcamp,c02e79ef
4,Where do I join the Slack channel?,data-engineering-zoomcamp,c02e79ef


In [31]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [32]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)


In [33]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)
